In [77]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq



# channelArr =[]
# pingNumArr =[]
sampleStorage = []
weightStorage = []


schema = pa.schema([("sample", pa.float64()), ("channel", pa.int8()), ("pingNum", pa.uint32())])
# table = pa.Table.from_arrays([col1, col2, col3], schema=schema)
big_Table = []
i = 0
arrayCounter = 1
with open("DANIA_17__Stave.jsf", 'rb') as file:
    # og num is 50717
    while(i<10):
        

        offset = np.fromfile(file,'int8',7)



        subSystem = np.fromfile(file,'int8',1)

        channel = np.fromfile(file,'int8',1)

        offset = np.fromfile(file,'int8',3)

        byteCount = np.fromfile(file,'uint32',1)

    
        if(subSystem == 0):
            np.fromfile(file,'int8', byteCount[0])

        if(subSystem == 20):
            np.fromfile(file,'int8', byteCount[0])
        if(subSystem == 21): 
            np.fromfile(file,'int8', byteCount[0])
        if(subSystem == 101): 
            np.fromfile(file,'int8', byteCount[0])
        if(subSystem == 102): 
            np.fromfile(file,'int8', byteCount[0])


        
        if(subSystem == 40):

            offset = np.fromfile(file,'int8',8)
            
            pingNum = np.fromfile(file,'uint32',1) 

            offset = np.fromfile(file,'int8',156)

            weight = np.fromfile(file,'int16',1) 
            
            offset = np.fromfile(file,'int8',70)
            
            numSamples = int((byteCount[0] - 240)/4)
            #! I explicetely casted to int because loop cant take in numpy.float64 values
            my_list = deque()
            
            tempArr = []
            tempArr2 = []
            for y in range((numSamples*2)):

                temp = (np.fromfile(file,'int16', 1))

                if y%2 == 0:
                    tempArr.append(temp[0])
                else:
                    tempArr2.append(temp[0])

            arrSize = len(tempArr)
            
            nuWeight = int(weight[0])
            nuWeight = 2**-(nuWeight)
            weightStorage = []
            # complexSample  []

            tempArr2 = np.array(tempArr2)
            tempArr = np.array(tempArr)

            tempArr2 = np.multiply(tempArr2, 1j)
            complexSample = np.add(tempArr, tempArr2)
            complexSample = np.multiply(complexSample, nuWeight)


            
            

            conjugateData = np.conj(complexSample)


            sample = np.multiply(complexSample, conjugateData)
            del tempArr
            del tempArr2
            del conjugateData
            del complexSample

            
            sample = np.real(sample)
            sample = sample[1699:]

            sampleSize = len(sample)

            channelArr =[]
            pingNumArr =[]

            channelArr = [channel[0]] * sampleSize
            pingNumArr = [pingNum[0]] * sampleSize

            

            sample = pa.array(sample)
            channelArr = pa.array(channelArr)
            pingNumArr = pa.array(pingNumArr)
            
            table = pa.Table.from_arrays([sample, channelArr, pingNumArr], schema=schema)
            big_Table.append(table)
            # big_Table = pa.concat_tables([table,table1])

            


        i+=1
                
                
table_concatenated = pa.concat_tables(big_Table)
pq.write_table(table_concatenated,'real_numbers_STAVE.parquet')




